In [68]:
%config IPCompleter.greedy=True


from sklearn.cluster import KMeans, DBSCAN, MeanShift
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from pylab import plot, show
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
import pandas as pd
import numpy as np


In [69]:

arr = np.genfromtxt('MT-human_clean2_genome_embeddings_sigmoid.csv', delimiter=',')
arr
# df = pd.read_csv('[Query-List of sequences]-MT-human_clean40_genome_embeddings_sigmoid.csv')

array([[0.55359, 0.58999, 0.46662, ..., 0.49621, 0.48489, 0.46733],
       [0.59408, 0.64203, 0.45641, ..., 0.50416, 0.41312, 0.46805],
       [0.61855, 0.55627, 0.51117, ..., 0.4889 , 0.45259, 0.49553],
       ...,
       [0.6109 , 0.61259, 0.4698 , ..., 0.49584, 0.48185, 0.42721],
       [0.58991, 0.61262, 0.48052, ..., 0.51071, 0.4756 , 0.49255],
       [0.57311, 0.62518, 0.45483, ..., 0.49195, 0.45402, 0.48419]])

In [70]:
# df

In [71]:
# arr = df.values

In [72]:
arr

array([[0.55359, 0.58999, 0.46662, ..., 0.49621, 0.48489, 0.46733],
       [0.59408, 0.64203, 0.45641, ..., 0.50416, 0.41312, 0.46805],
       [0.61855, 0.55627, 0.51117, ..., 0.4889 , 0.45259, 0.49553],
       ...,
       [0.6109 , 0.61259, 0.4698 , ..., 0.49584, 0.48185, 0.42721],
       [0.58991, 0.61262, 0.48052, ..., 0.51071, 0.4756 , 0.49255],
       [0.57311, 0.62518, 0.45483, ..., 0.49195, 0.45402, 0.48419]])

In [73]:
len(arr)

690

In [74]:
len(arr[0])

1000

In [75]:
def evaluate_kmeans():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        model = KMeans(n_clusters=i)
        model.fit(arr)
        predicted_labels = model.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
    return max_score, predicted_labels_final, clusters

In [77]:
def evaluate_gmm():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='kmeans')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='random')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    return max_score, predicted_labels_final, clusters

In [78]:
def evaluate_dbscan():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(1, 10000, 1):
        for j in range(50, 200, 10):
            eps = i*0.5
            dbscan = DBSCAN(eps=i,min_samples=j)
            dbscan.fit(arr)
            predicted_labels = dbscan.labels_
#             print (eps)
#             print (j)
#             print (predicted_labels)
            
            labels, count = np.unique(predicted_labels, return_counts= True)
            if (len(labels) == 1):
                continue
            silhouette_avg = silhouette_score(arr, predicted_labels)
            silhouette_avg
            if(max_score < silhouette_avg):
                max_score = silhouette_avg
                predicted_labels_final = predicted_labels
                clusters = labels
                
    return max_score, predicted_labels_final, clusters

In [79]:
score_kmeans, predicted_label_kmeans, cluster_kmeans = evaluate_kmeans()


In [ ]:
score_gmm, predicted_label_gmm, cluster_gmm = evaluate_gmm()

In [62]:
score_dbscan, predicted_label_dbscan, clusters_dbscan = evaluate_dbscan()

KeyboardInterrupt: 

In [67]:
 print ('KMeans results: ')
print ('Silhoeutte Score :', score_kmeans)
print ('Number of clusters formed: ', cluster_kmeans)

KMeans results: 
Silhoeutte Score : 0.18664627403758616
Number of clusters formed:  4


###### predicted_labels_Gmm

In [66]:
print ('GMM results: ')
print ('Silhoeutte Score :', score_gmm)
print ('Number of clusters formed: ', cluster_gmm)

GMM results: 
Silhoeutte Score : 0.18664627403758616
Number of clusters formed:  4


In [60]:
score_dbscan, predicted_label_dbscan
print ('DBSCAN results: ')
print ('Silhoeutte Score :', score_dbscan)
print ('Number of clusters formed: ', cluster_dbscan)

(-1, [])